# Define Constants

In [ ]:
from random import randint
from IPython.display import clear_output
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
# %matplotlib inline

input_path = './chest_Xray/'
img_dims = 150
epochs = 30
batch_size = 64


def process_data(img_dims, batch_size):
    test_data = []
    test_labels = []

    for cond in ['/NORMAL/', '/PNEUMONIA/']:
        for img in (os.listdir(input_path + 'test' + cond)):
            img = plt.imread(input_path+'test'+cond+img)
            img = cv2.resize(img, (img_dims, img_dims))
            img = np.dstack([img, img, img])
            img = img.astype('float32') / 255
            if cond == '/NORMAL/':
                label = 0
            elif cond == '/PNEUMONIA/':
                label = 1
            test_data.append(img)
            test_labels.append(label)
        
    test_data = np.array(test_data)
    test_labels = np.array(test_labels)

    return test_data, test_labels

# Import Data

In [ ]:
n_normal = len(os.listdir(input_path + 'test' + '/NORMAL'))
n_infect = len(os.listdir(input_path + 'test' + '/PNEUMONIA'))
print('Set: {}, normal images: {}, pneumonia images: {}'.format(
    'test', n_normal, n_infect))

test_data, test_labels = process_data(
    img_dims, batch_size)

# Import pre-trained Models

In [ ]:
from tensorflow.keras.models import load_model

modelList = []
for fold in os.listdir('./saved_models_v2/'):
    m = load_model('./saved_models_v2/' + fold)
    modelList.append([m, fold])
    print("Imported: {}/{}".format(len(modelList),
          len(os.listdir('./saved_models_v2/'))))
    clear_output(wait=True)
print("Import OK")

# Evaluate Models

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

i = 0
max_index_acc = 0
max_index_f = 0
max_name_acc = ""
max_name_f = ""
max_acc = 0
max_f = 0
for model in modelList:
    print("{}/{} - {}".format(i + 1, len(modelList), model[1]), end=" - ")
    loss, acc = model[0].evaluate(
        test_data, test_labels, verbose=2)
    # preds = modelList[max_index_acc][0].predict(test_data)

    # cm = confusion_matrix(test_labels, np.round(preds))
    # tn, fp, fn, tp = cm.ravel()
    # precision = tp/(tp+fp)*100
    # recall = tp/(tp+fn)*100
    # f = 2 * precision * recall/(precision + recall)
    if (acc > max_acc):
        max_acc = acc
        max_name_acc = model[1]
        max_index_acc = i
    # if (f > max_f):
    #     max_f = f
    #     max_name_f = model[1]
    #     max_index_f = i
    i += 1

    # print(model[1] + ' - Accuracy: {:5.2f}%'.format(100 * acc))
print("\nBest acc: {} at index: {}\n".format(max_name_acc, max_index_acc))
# print("Best F1: {} at index: {}".format(max_name_f, max_index_f))

# Display most accurate model info

In [ ]:
import seaborn as sns

preds = modelList[max_index_acc][0].predict(test_data)

acc = accuracy_score(test_labels, np.round(preds))*100
cm = confusion_matrix(test_labels, np.round(preds))
tn, fp, fn, tp = cm.ravel()

print("--- DATA FOR MAX ACC AT {} ---\n".format(modelList[max_index_acc][1]))

print('TEST METRICS ----------------------')
precision = tp/(tp+fp)*100
recall = tp/(tp+fn)*100
f = 2 * precision * recall/(precision + recall)
print('Accuracy: {}%'.format(acc))
print('Precision: {}%'.format(precision))
print('Recall: {}%'.format(recall))
print('F1-score: {}'.format(f))

print('\nCONFUSION MATRIX ------------------')
print(cm)
group_names = ['True Neg', 'False Pos', 'False Neg', 'True Pos']
group_percentages = ["{0:.2%}".format(value)
                     for value in cm.flatten()/np.sum(cm)]
labels = [f"{v1}\n{v3}" for v1, v3 in zip(group_names, group_percentages)]
labels = np.asarray(labels).reshape(2, 2)
sns.heatmap(cm, annot=labels, fmt='', cmap='Blues')

# Display all model's confusion matrix

In [ ]:
for model in modelList:
    preds = model[0].predict(test_data)

    acc = accuracy_score(test_labels, np.round(preds))*100
    cm = confusion_matrix(test_labels, np.round(preds))
    tn, fp, fn, tp = cm.ravel()

    precision = tp/(tp+fp)*100
    recall = tp/(tp+fn)*100
    f = 2 * precision * recall/(precision + recall)

    group_names = ['True Neg', 'False Pos', 'False Neg', 'True Pos']
    group_percentages = ["{0:.2%}".format(value)
                        for value in cm.flatten()/np.sum(cm)]
    labels = [f"{v1}\n{v3}" for v1, v3 in zip(group_names, group_percentages)]
    labels = np.asarray(labels).reshape(2, 2)
    sns.heatmap(cm, annot=labels, fmt='', cmap='Blues')
    t = model[1].split('_')
    title = "epochs: {} - batch: {} - acc: {}".format(t[0], t[2].split('b')[1], t[3].split('a')[1])
    plt.title(title)
    plt.show()